In [1]:
import numpy as np
import  pandas as pd

In [2]:
df = pd.read_csv("../dataset.csv", header=None)
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float16
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string

In [3]:
dataset = dataset.reshape(dataset.shape[0], 13, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (583554, 13, 16, 1)
Labels shape: (583554,)
Input shape: (13, 16, 1)


In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.05, random_state=42, stratify=labels_set)

In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

2025-03-19 19:58:49.173943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742410729.190976   37700 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742410729.195974   37700 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 19:58:49.213315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/fridh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. 

In [6]:
def indexOfMax(arr):
    return np.argmax(arr)

In [7]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [8]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 115200 baud.


In [9]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [10]:
calc_accuracy(np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]]), [1, 0, 1])

0.6667

In [11]:
x = x_test
y = y_test[0:len(x)]
print(x.shape)

(29178, 13, 16, 1)


In [12]:
import time
import struct

results = []
total = len(x)

# Send data
for i in range(total):
    data = x[i].flatten()
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
    time.sleep(1)
    # Read response
    response = ser.readline().decode('utf-8').strip()
    results.append(response)
    print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


ValueError: invalid literal for int() with base 10: ''

In [15]:
# Close the serial port
ser.close()